# Earth Engine Datalab Initialization

This notebook demonstrates how to configure a new Google Cloud Datalab instance for use with Earth Engine. You should only need to do this once when you first configure your instance.

## 1. Install Earth Engine SDK and related libraries

First we install a variety of required and useful libraries, including the Earth Engine SDK itself. This may take a couple of mintues, and will generate lots of output which you can usually ignore.

In [1]:
%%bash
apt-get update
apt-get install -y libffi-dev libssl-dev python-dev python-pip python-imaging python-gdal
pip install cryptography google-api-python-client earthengine-api

bash: line 1: apt-get: command not found
bash: line 2: apt-get: command not found


## 2. Authorize access to the Earth Engine API

This walks through the same flow that is used by the "`earthengine authenticate`" command, but it works even when you do not have easy access to an interactive shell on the remote host.  These should become available as helper functions in the Earth Engine client library, but for now you can still do it by hand.

Start by importing the Python libraries that we're going to use to configure the Earth Engine credentials.

In [2]:
import ee
import errno
import IPython
import json
import os
import urllib
import urllib2

These helper functions implement the authentication flow.  

In [3]:
def get_authorization_url():
  # This redirect_uri prompts you to copy and paste a code after successful authorization.
  return 'https://accounts.google.com/o/oauth2/auth?' + urllib.urlencode({
      'client_id': ee.oauthinfo.OAuthInfo.CLIENT_ID,
      'scope': ee.oauthinfo.OAuthInfo.SCOPE,
      'redirect_uri': 'urn:ietf:wg:oauth:2.0:oob',
      'response_type': 'code',
  })
  
def get_refresh_token(auth_code):
  args = {
    'code': auth_code,
    'client_id': ee.oauthinfo.OAuthInfo.CLIENT_ID,
    'client_secret': ee.oauthinfo.OAuthInfo.CLIENT_SECRET,
    'redirect_uri': 'urn:ietf:wg:oauth:2.0:oob',
    'grant_type': 'authorization_code',
  }
  response = urllib2.urlopen('https://accounts.google.com/o/oauth2/token',
                             urllib.urlencode(args)).read()
  return json.loads(response)['refresh_token']

def save_token(refresh_token):
  credentials_path = ee.oauthinfo.OAuthInfo.credentials_path()
  try:
    os.makedirs(os.path.dirname(credentials_path))
  except OSError, e:
    if e.errno != errno.EEXIST:
      raise
  json.dump({'refresh_token': refresh_token}, open(credentials_path, 'w'))
  print 'Successfully saved authorization to %s' % credentials_path

Running this next command will print a URL that you can click on to begin the authentication flow.  Copy the code that it generates and come back here for the next step.

In [4]:
print(get_authorization_url())

https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.read_write&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com


Finally, **paste the auth code that you were given** into this next command (in place of  "`YOUR_AUTH_TOKEN_GOES_HERE`") and run it to save your credentials on this Datalab instance.

In [6]:
save_token(get_refresh_token('https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.read_write&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com'))

HTTPError: HTTP Error 400: Bad Request

## 3. Take it for a spin
Finally, test out our new credentials by doing something with Earth Engine. This should display a simple thumbnail of global SRTM terrain data.

In [7]:
ee.Initialize()
srtm = ee.Image('USGS/SRTMGL1_003')
srtm_vis = srtm.visualize(min=0, max=5000, gamma=1.6)
srtm_thumb = ee.data.getThumbnail({'image':srtm_vis.serialize()})
IPython.display.display(IPython.display.Image(srtm_thumb))

HttpAccessTokenRefreshError: invalid_grant

In [8]:
!ls

20141103-World-Origin.mov
26fb4HJKeWtAAAAAElFTkSuQmCC.png
A1--THINGS TO DO--9 June 2016.docx
Behrenfeld_Falkowski_1997 (1).pdf
Chlorophyll_2015_01.tiff
Earth Engine Datalab Initialization.ipynb
Earth Engine and TensorFlow.ipynb
Global Density Raster High Res.py
Hobet_timelapse.mp4
Jobs
MY1DMM_CHLORA_2015-01-01_rgb_3600x1800.FLOAT.TIFF
Rast_high_res.py
Screen Shot 2016-06-13 at 10.00.58 PM.png
Screen Shot 2016-06-13 at 10.03.07 PM.png
Screen Shot 2016-06-13 at 10.11.10 PM.png
Screen Shot 2016-06-13 at 10.19.11 PM.png
Screen Shot 2016-06-13 at 10.47.14 PM.png
Screen Shot 2016-06-13 at 10.48.02 PM.png
Screen Shot 2016-06-13 at 10.51.39 PM.png
Screen Shot 2016-06-13 at 10.51.48 PM.png
Screen Shot 2016-06-13 at 10.52.08 PM.png
Screen Shot 2016-06-13 at 2.25.54 PM (2).png
Screen Shot 2016-06-13 at 2.25.54 PM.png
Screen Shot 2016-06-13 at 2.25.57 PM (2).png
Screen Shot 2016-06-13 at 2.25.57 PM.png
Screen Shot 2016-06-13 at 9.48.57 PM.png
Screen Shot 2016-06-14 at 1.37.06 PM.png
Screen Shot 20